Observer Pattern
===

> 定义了对象之间的一对多依赖，这样一来，当一个对象改变状态时，它的所有依赖者都会收到通知并自动更新

In [1]:
import uuid
from collections import namedtuple, defaultdict


class BasePublisher:

    """基础的发布者类
    提供发布者应有的基础功能，可以继承并覆盖为自定义的方法"""

    _TASK_TUPLE = namedtuple('EventListener', ['event', 'callback', 'context'])
    _EVENT_LISTENERS = {}
    _TASK_IDS = {}

    def _create_new_event(self, event):
        if event in self._EVENT_LISTENERS:
            return

        self._EVENT_LISTENERS[event] = defaultdict(dict)

    def generate_task_id(self):
        return uuid.uuid1()

    def generate_task_tuple(self, event, callback, context):
        return self._TASK_TUPLE(event=event, callback=callback, context=context)

    def _add_new_task(self, event, task_id, task_tuple):
        assert task_id not in self._TASK_IDS
        self._create_new_event(event)
        self._EVENT_LISTENERS[event][task_id] = task_tuple
        self._TASK_IDS[task_id] = event

    def _remove_task(self, task_id):
        assert task_id in self._TASK_IDS
        event = self._TASK_IDS[task_id]
        del self._EVENT_LISTENERS[event][task_id]
        del self._TASK_IDS[task_id]

    def register(self, event, callback, context=None):
        """注册事件和回调函数

        Parameters
        ----------
        event : str
            事件名
        callback : callback(event, event_data, context)
            事件回调函数，依次传入事件名，事件数据和注册上下文
        context : 事件上下文，会在触发回调时传入
        """
        task_id = self.generate_task_id()
        task_tuple = self.generate_task_tuple(event, callback, context)
        self._add_new_task(event, task_id, task_tuple)
        return task_id

    def unregister(self, task_id):
        """注销"""
        self._remove_task(task_id)
        return task_id

    def trigger(self, event, event_data):
        """触发器
        每当有事件发生时，应该触发该函数，该函数负责调用注册的监听函数
        """
        for task_tuple in self._EVENT_LISTENERS:
            try:
                callback = task_tuple.callback
                context = task_tuple.context
                callback(event, event_data, context)
            except Exception as err:
                self.exception(err, event, event_data, task_tuple)

    def exception(self, err, event, event_data, task_tuple):
        pass
